<a href="https://colab.research.google.com/github/Firojpaudel/GenAI-Chronicles/blob/main/GANs/GAN_With_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Trying to implement GAN using PyTorch**: MNIST Dataset
---

In [1]:
##@ Imports

import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms

#### 1. Creating the Generator

In [2]:
##@ First lets create the generator nn

class Generator(nn.Module):
  def __init__(self, latent_dim, img_shape):
    super(Generator, self).__init__()
    self.img_shape = img_shape

    def block(in_features, out_features, normalize= True):
      layers = [nn.Linear(in_features, out_features)]
      if normalize:
        layers.append(nn.BatchNorm1d(out_features, momentum=0.8))
      layers.append(nn.LeakyReLU(0.2, inplace= True))
      return layers

    self.model= nn.Sequqntial(
        *block(latent_dim, 128, normalize = False),
        *block(128, 256),
        *block(256, 512),
        *block(512, 1024),
        nn.Linear(1024, int(np.prod(img_shape))),  #Trying to match with the dimension of the image and then we apply the activation function
        nn.Tanh()
    )


  def forward(self, z):
    img = self.model(z)
    return img.view(img.size(0), *self.img_shape)

##### **Explaining the code above:** _Generator_

---


Okay, so first lets explain the block function and the need for it:

If we did not use block function the code snippet for `self.model` would look like:

```python
 self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),  # First layer
            nn.LeakyReLU(0.2, inplace=True),  # Activation
            nn.Linear(128, 256),  # Second layer
            nn.BatchNorm1d(256, momentum=0.8),  # Batch normalization
            nn.LeakyReLU(0.2, inplace=True),  # Activation
            nn.Linear(256, 512),  # Third layer
            nn.BatchNorm1d(512, momentum=0.8),  # Batch normalization
            nn.LeakyReLU(0.2, inplace=True),  # Activation
            nn.Linear(512, 1024),  # Fourth layer
            nn.BatchNorm1d(1024, momentum=0.8),  # Batch normalization
            nn.LeakyReLU(0.2, inplace=True),  # Activation
            nn.Linear(1024, int(np.prod(img_shape))),  # Output layer
            nn.Tanh()  # Final activation function
        )

```

which is obviously very tough to handle. So we just created the function `"block"` which opens up in the model created.

> **_Note:_** \
When you call `*block(...)` inside `nn.Sequential`, the asterisk _unpacks the list of layers returned by the block function_. Without the asterisk, the code would pass the whole list as a single element, which would cause an error.

The same goes for ` *self.img_shape`. It's just unpacking the image dimensions



### 2. Creating the discriminator

In [3]:
##@ Now Creating the discriminator:

class Discriminator(nn.Module):
  def __init__(self, img_shape):
    super(Discriminator, self).__init__()

    self.model= nn.Sequential(
        nn.Linear(int(np.prod(img_shape)), 512),
        nn.LeakyReLU(0.2, inplace= True),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace= True),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

    def forward(self, img):
      img_flat = img.view(img.size(0), -1)
      validity = self.model(img_flat)
      return validity

##### **Explaining the code above:** _Discriminator_

---

Okay, so here, its pretty straight forward. We create the model which has the final activation funciton of `sigmoid` and this simply classifies as real or fake.

> 1 being the absolute real image classification and 0 being the absolute fake image classification

Also lets discuss about the `img.view`:

Explaining with the example case,

Let's say we have a batch of **3 RGB images, each of size 64x64**:

- `img.shape` would be $(3, 3, 64, 64)$ ie. $(\text{batch_size}, \text{channels}, \text{height}, \text{width})$.

`img.view(img.size(0), -1)` would do the following:
- `img.size(0)` is 3 (the batch size).
- `-1` calculates $3 * 64 * 64 = 12288$.

The resulting shape would be (3, 12288)

#### 3. Defining the Loss Function

In [4]:
##@ Loss Function

adversarial_loss = torch.nn.BCELoss() #The adversarial loss is simply the Binary Cross Entropy Loss

#### 4. Initialize the Generator and Disriminator